In [42]:
import collections
import pandas as pd
import datetime
import numpy as np
import json
import codecs

In [43]:
data_fr = pd.read_csv('../data/data_fr.csv', sep=';', usecols=np.arange(2,35))
corres_ICD10 = pd.read_csv('../data/corres_ICD10.csv', sep=';',usecols=[1,2,3,4,5])
corres_geo = pd.read_csv('../data/corres_geo.csv', sep=';', usecols=[1,2,3,4])

In [44]:
corres_geo.head()

,niveau_0,niveau_1,niveau_2,region
0,FR,FR,FR,France
1,FR,FR1,FR1,ile de France
2,FR,FR1,FR10,ile de France
3,FR,FR2,FR2,Bassin Parisien (NUTS 2013)
4,FR,FR2,FR21,Champagne-Ardenne (NUTS 2013)


In [45]:
corres_ICD10.head()

,niveau_0,niveau_1,niveau_2,niveau_3,maladies
0,A-R_V-Y,A-R_V-Y,A-R_V-Y,A-R_V-Y,Toutes causes de mortalite (A00-Y89) sauf S00-T98
1,A-R_V-Y,A_U,A_U,A_U,Causes interne de morbidite et mortalite
2,A-R_V-Y,A_U,A_B,A_B,Certaines maladies infectieuses et parasitaire...
3,A-R_V-Y,A_U,A_B,A15-A19_B90,Tuberculose
4,A-R_V-Y,A_U,A_B,B15-B19_B942,Hepatite virale et sequelles d'hepatite virale


In [46]:
# Nettoyage des données:
for col in data_fr.columns[11:]:
    data_fr[col] = data_fr[col].str.strip().replace({":":"NaN", ": z":"NaN"}).astype('float64')

In [47]:
data_fr.head()

,sex,age,geo_niv_0,geo_niv_1,geo_niv_2,region,icd10_niv_0,icd10_niv_1,icd10_niv_2,icd10_niv_3,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994
0,F,TOTAL,FR,FR,FR,France,A-R_V-Y,A_U,A_B,A15-A19_B90,...,1.4,1.5,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F,TOTAL,FR,FR1,FR10,ile de France,A-R_V-Y,A_U,A_B,A15-A19_B90,...,1.3,1.2,1.4,1.5,1.7,1.5,1.4,1.4,1.6,1.4
2,F,TOTAL,FR,FR2,FR21,Champagne-Ardenne (NUTS 2013),A-R_V-Y,A_U,A_B,A15-A19_B90,...,1.2,1.2,2.0,0.7,2.2,0.7,0.9,2.2,1.3,2.8
3,F,TOTAL,FR,FR2,FR22,Picardie (NUTS 2013),A-R_V-Y,A_U,A_B,A15-A19_B90,...,0.9,1.6,1.6,1.4,1.1,1.3,1.3,1.2,1.0,1.7
4,F,TOTAL,FR,FR2,FR23,Haute-Normandie (NUTS 2013),A-R_V-Y,A_U,A_B,A15-A19_B90,...,1.5,1.4,2.7,1.4,2.5,1.4,1.8,1.1,2.1,2.1


In [7]:
data_fr.drop(['geo_niv_2', 'icd10_niv_3', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994'], inplace=True, axis=1)

In [8]:
data_fr.to_csv('data_fr2.csv')

In [37]:
df = data_fr.pivot_table(index=data_fr.columns[2:9].to_list(),
                   values=data_fr.columns[9:].to_list(),
                   columns=data_fr.columns[:2].to_list())
df.to_json('arbo_1.json')

In [55]:
df

2005  \
sex                                                                                                                                                    F   
age                                                                                                                                                TOTAL   
geo_niv_0 geo_niv_1 region                                 icd10_niv_0 icd10_niv_1 icd10_niv_2 maladies                                                    
FR        FR        France                                 A-R_V-Y     A-R_V-Y     A-R_V-Y     Toutes causes de mortalite (A00-Y89) sauf S00-T98   804.7   
                                                                       A_U         A_B         Autres certaines maladies infectieuses et paras...   12.5   
                                                                                               Certaines maladies infectieuses et parasitaires...   15.6   
                                                                                               Hepatite virale et sequelles d'hepatite virale        1.1   
                                                                                               Maladies dues au virus de l'immunodeficience hu...    0.7   
                                                                                               Tuberculose                                           1.3   
                                                                                   C_D         Tumeurs                                             197.7   
                                                                                   C_D-C       Autre tumeurs malignes des tissus lymphoede. he...    4.4   
                                                                                               Autres tumeurs malignes (partie restante de C00...   28.8   
                                                                                               Leucemie                                              7.6   
                                                                                               Maladie de Hodgkin et lymphomes                       6.5   
                                                                                               Melanome malin de la peau                             2.2   
                                                                                               Tumeur maligne d'autres parties de l'uterus           7.0   
                                                                                               Tumeur maligne de l'estomac                           5.7   
                                                                                               Tumeur maligne de l'oesophage                         2.1   
                                                                                               Tumeur maligne de l'ovaire                           10.3   
                                                                                               Tumeur maligne de la levre. de la cavite buccal...    2.3   
                                                                                               Tumeur maligne de la prostate                         NaN   
                                                                                               Tumeur maligne de la thyroede                         0.8   
                                                                                               Tumeur maligne de la trachee et des bronches et...   18.4   
                                                                                               Tumeur maligne de la vessie                           3.5   
                                                                                               Tumeur maligne du celon. de la jonction recto-s...   24.5   
                                                                                            

In [56]:
df = data_fr.pivot_table(index=data_fr.columns[2:9].to_list(),
                   values=data_fr.columns[9:].to_list(),
                   columns=data_fr.columns[:1].to_list())
df.to_json('arbo_2.json')

In [57]:
df.head()

2005  \
sex                                                                                                                          F   
geo_niv_0 geo_niv_1 region icd10_niv_0 icd10_niv_1 icd10_niv_2 maladies                                                          
FR        FR        France A-R_V-Y     A-R_V-Y     A-R_V-Y     Toutes causes de mortalite (A00-Y89) sauf S00-T98   1340.231818   
                                       A_U         A_B         Autres certaines maladies infectieuses et paras...    21.100000   
                                                               Certaines maladies infectieuses et parasitaires...    25.427273   
                                                               Hepatite virale et sequelles d'hepatite virale         1.563636   
                                                               Maladies dues au virus de l'immunodeficience hu...     0.622727   

                                                                                                                                \
sex                                                                                                                          M   
geo_niv_0 geo_niv_1 region icd10_niv_0 icd10_niv_1 icd10_niv_2 maladies                                                          
FR        FR        France A-R_V-Y     A-R_V-Y     A-R_V-Y     Toutes causes de mortalite (A00-Y89) sauf S00-T98   2024.031818   
                                       A_U         A_B         Autres certaines maladies infectieuses et paras...    29.413636   
                                                               Certaines maladies infectieuses et parasitaires...    37.068182   
                                                               Hepatite virale et sequelles d'hepatite virale         2.195455   
                                                               Maladies dues au virus de l'immunodeficience hu...     1.881818   

                                                                                                                                \
sex                                                                                                                          T   
geo_niv_0 geo_niv_1 region icd10_niv_0 icd10_niv_1 icd10_niv_2 maladies                                                          
FR        FR        France A-R_V-Y     A-R_V-Y     A-R_V-Y     Toutes causes de mortalite (A00-Y89) sauf S00-T98   1608.140909   
                                       A_U         A_B         Autres certaines maladies infectieuses et paras...    24.181818   
                                                               Certaines maladies infectieuses et parasitaires...    29.922727   
                                                               Hepatite virale et sequelles d'hepatite virale         1.831818   
                                                               Maladies dues au virus de l'immunodeficience hu...     1.227273   

                                                                                                                          2006  \
sex                                                                                                                          F   
geo_niv_0 geo_niv_1 region icd10_niv_0 icd10_niv_1 icd10_niv_2 maladies                                                          
FR        FR        France A-R_V-Y     A-R_V-Y     A-R_V-Y     Toutes causes de mortalite (A00-Y89) sauf S00-T98   1251.281818   
                                       A_U         A_B         Autres certaines maladies infectieuses et paras...    20.336364   
                                                               Certaines maladies infectieuses et parasitaires...    24.031818   
                                                               Hepatite virale et sequelles d'hepatite virale         1.409091   
                                                               Maladies dues au virus de 

In [58]:
# ============ Dictionnaires imbriqués ===========
# ================================================

# def nested_dict():
#     return collections.defaultdict(nested_dict)
# result = nested_dict()

# for row in df.itertuples():
#     result[row.Index[0]][row.Index[1]][row.Index[2]][row.Index[3]][row.Index[4]][row.Index[5]] = row._1
    
# out = json.dumps(result)

# with open('test.json', 'w') as f:
#     f.write(out)